<a href="https://colab.research.google.com/github/EOKELLO/IP-WEEK1-CORE/blob/master/Moringa_Data_Science_Core_W8_Independent_Project_2019_07_Elizabeth_Okello_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Research Objective**

Build a model that determines whether or not the patient's symptoms indicate that the patient has hypothyroid

# Metric for success ** 

1. Build a model with an accuracy of  above 80% 

2. Be able to identify the most influential variables in making the prediction

# The **context**

we shall use the healthcare dataset collected by Nairobi Hospital that focused on Thyroid patients.   

# **Experimental design**

1.Load the data

2. clean the data

3.Convert the data to the appropriate data types for mmodelling

4.model and evaluate the model

5.Tune the model

6.Identify the features with the most say

7.Reccommend the best model giving reasons

# Appropriateness of **Data

The data was appropriate in terms of relevance and volume.

# **Importing our Libraries**

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# **Loading the Dataset**

In [2]:
data = pd.read_csv('hypothyroid.csv')
data.head()

,status,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,thyroid_surgery,query_hypothyroid,query_hyperthyroid,pregnant,sick,tumor,lithium,goitre,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG
0,hypothyroid,72,M,f,f,f,f,f,f,f,f,f,f,f,y,30,y,0.6,y,15,y,1.48,y,10,n,?
1,hypothyroid,15,F,t,f,f,f,f,f,f,f,f,f,f,y,145,y,1.7,y,19,y,1.13,y,17,n,?
2,hypothyroid,24,M,f,f,f,f,f,f,f,f,f,f,f,y,0,y,0.2,y,4,y,1,y,0,n,?
3,hypothyroid,24,F,f,f,f,f,f,f,f,f,f,f,f,y,430,y,0.4,y,6,y,1.04,y,6,n,?
4,hypothyroid,77,M,f,f,f,f,f,f,f,f,f,f,f,y,7.3,y,1.2,y,57,y,1.28,y,44,n,?


In [3]:
data['sex'].value_counts()

F    2182
M     908
?      73
Name: sex, dtype: int64

In [0]:
data=data.replace(['?'], np.nan)

In [5]:
data.shape

(3163, 26)

In [6]:
data.isnull().sum()

status                          0
age                           446
sex                            73
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
thyroid_surgery                 0
query_hypothyroid               0
query_hyperthyroid              0
pregnant                        0
sick                            0
tumor                           0
lithium                         0
goitre                          0
TSH_measured                    0
TSH                           468
T3_measured                     0
T3                            695
TT4_measured                    0
TT4                           249
T4U_measured                    0
T4U                           248
FTI_measured                    0
FTI                           247
TBG_measured                    0
TBG                          2903
dtype: int64

In [0]:
# Creating a list of numerical columns
numeric=['age', 'FTI', 'T3', 'T4U', 'TSH', 'TT4']
for i in data.columns:
   if i in numeric:
       data[i]=pd.to_numeric(data[i])
   else:
        data[i]=data[i].astype('category')



In [0]:
#for column in data[['age', 'TSH', 'T3', 'TT4','T4U','FTI']]:
  #  mode = data[column].mode()
   # data[column] = data[column].fillna(mode)

In [0]:
data=data.fillna(data.mean())

In [10]:
data.isnull().sum()

status                          0
age                             0
sex                            73
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
thyroid_surgery                 0
query_hypothyroid               0
query_hyperthyroid              0
pregnant                        0
sick                            0
tumor                           0
lithium                         0
goitre                          0
TSH_measured                    0
TSH                             0
T3_measured                     0
T3                              0
TT4_measured                    0
TT4                             0
T4U_measured                    0
T4U                             0
FTI_measured                    0
FTI                             0
TBG_measured                    0
TBG                          2903
dtype: int64

In [11]:
data['sex'].value_counts()

F    2182
M     908
Name: sex, dtype: int64

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3163 entries, 0 to 3162
Data columns (total 26 columns):
status                       3163 non-null category
age                          3163 non-null float64
sex                          3090 non-null category
on_thyroxine                 3163 non-null category
query_on_thyroxine           3163 non-null category
on_antithyroid_medication    3163 non-null category
thyroid_surgery              3163 non-null category
query_hypothyroid            3163 non-null category
query_hyperthyroid           3163 non-null category
pregnant                     3163 non-null category
sick                         3163 non-null category
tumor                        3163 non-null category
lithium                      3163 non-null category
goitre                       3163 non-null category
TSH_measured                 3163 non-null category
TSH                          3163 non-null float64
T3_measured                  3163 non-null category
T3         

In [13]:
data.columns

Index(['status', 'age', 'sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'thyroid_surgery', 'query_hypothyroid',
       'query_hyperthyroid', 'pregnant', 'sick', 'tumor', 'lithium', 'goitre',
       'TSH_measured', 'TSH', 'T3_measured', 'T3', 'TT4_measured', 'TT4',
       'T4U_measured', 'T4U', 'FTI_measured', 'FTI', 'TBG_measured', 'TBG'],
      dtype='object')

In [14]:
data.head()

,status,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,thyroid_surgery,query_hypothyroid,query_hyperthyroid,pregnant,sick,tumor,lithium,goitre,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG
0,hypothyroid,72.0,M,f,f,f,f,f,f,f,f,f,f,f,y,30.0,y,0.6,y,15.0,y,1.48,y,10.0,n,NaN
1,hypothyroid,15.0,F,t,f,f,f,f,f,f,f,f,f,f,y,145.0,y,1.7,y,19.0,y,1.13,y,17.0,n,NaN
2,hypothyroid,24.0,M,f,f,f,f,f,f,f,f,f,f,f,y,0.0,y,0.2,y,4.0,y,1.00,y,0.0,n,NaN
3,hypothyroid,24.0,F,f,f,f,f,f,f,f,f,f,f,f,y,430.0,y,0.4,y,6.0,y,1.04,y,6.0,n,NaN
4,hypothyroid,77.0,M,f,f,f,f,f,f,f,f,f,f,f,y,7.3,y,1.2,y,57.0,y,1.28,y,44.0,n,NaN


In [15]:
#converting categorical data to dummy variables
categorical = ['sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'thyroid_surgery', 'query_hypothyroid',
       'query_hyperthyroid', 'pregnant', 'sick', 'tumor', 'lithium', 'goitre',
       'TSH_measured', 'T3_measured',  'TT4_measured', 
       'T4U_measured',  'FTI_measured', 'TBG_measured']
dummy = pd.get_dummies(data[categorical])
dummy.head(2)

,sex_F,sex_M,on_thyroxine_f,on_thyroxine_t,query_on_thyroxine_f,query_on_thyroxine_t,on_antithyroid_medication_f,on_antithyroid_medication_t,thyroid_surgery_f,thyroid_surgery_t,query_hypothyroid_f,query_hypothyroid_t,query_hyperthyroid_f,query_hyperthyroid_t,pregnant_f,pregnant_t,sick_f,sick_t,tumor_f,tumor_t,lithium_f,lithium_t,goitre_f,goitre_t,TSH_measured_n,TSH_measured_y,T3_measured_n,T3_measured_y,TT4_measured_n,TT4_measured_y,T4U_measured_n,T4U_measured_y,FTI_measured_n,FTI_measured_y,TBG_measured_n,TBG_measured_y
0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0
1,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0


In [16]:
#combining the dummy variables wih the initial dataset
data = pd.concat([data,dummy], axis=1)
data.head()

,status,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,thyroid_surgery,query_hypothyroid,query_hyperthyroid,pregnant,sick,tumor,lithium,goitre,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,sex_F,sex_M,on_thyroxine_f,on_thyroxine_t,query_on_thyroxine_f,query_on_thyroxine_t,on_antithyroid_medication_f,on_antithyroid_medication_t,thyroid_surgery_f,thyroid_surgery_t,query_hypothyroid_f,query_hypothyroid_t,query_hyperthyroid_f,query_hyperthyroid_t,pregnant_f,pregnant_t,sick_f,sick_t,tumor_f,tumor_t,lithium_f,lithium_t,goitre_f,goitre_t,TSH_measured_n,TSH_measured_y,T3_measured_n,T3_measured_y,TT4_measured_n,TT4_measured_y,T4U_measured_n,T4U_measured_y,FTI_measured_n,FTI_measured_y,TBG_measured_n,TBG_measured_y
0,hypothyroid,72.0,M,f,f,f,f,f,f,f,f,f,f,f,y,30.0,y,0.6,y,15.0,y,1.48,y,10.0,n,NaN,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0
1,hypothyroid,15.0,F,t,f,f,f,f,f,f,f,f,f,f,y,145.0,y,1.7,y,19.0,y,1.13,y,17.0,n,NaN,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0
2,hypothyroid,24.0,M,f,f,f,f,f,f,f,f,f,f,f,y,0.0,y,0.2,y,4.0,y,1.00,y,0.0,n,NaN,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0
3,hypothyroid,24.0,F,f,f,f,f,f,f,f,f,f,f,f,y,430.0,y,0.4,y,6.0,y,1.04,y,6.0,n,NaN,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0
4,hypothyroid,77.0,M,f,f,f,f,f,f,f,f,f,f,f,y,7.3,y,1.2,y,57.0,y,1.28,y,44.0,n,NaN,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0


In [0]:
#we then drop the values we have converted to dummies plus the TBG column as it still has missing values
data.drop(['sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'thyroid_surgery', 'query_hypothyroid',
       'query_hyperthyroid', 'pregnant', 'sick', 'tumor', 'lithium', 'goitre',
       'TSH_measured', 'T3_measured',  'TT4_measured', 
       'T4U_measured',  'FTI_measured', 'TBG_measured','TBG'], axis=1, inplace=True)


In [18]:
data.columns

Index(['status', 'age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI', 'sex_F', 'sex_M',
       'on_thyroxine_f', 'on_thyroxine_t', 'query_on_thyroxine_f',
       'query_on_thyroxine_t', 'on_antithyroid_medication_f',
       'on_antithyroid_medication_t', 'thyroid_surgery_f', 'thyroid_surgery_t',
       'query_hypothyroid_f', 'query_hypothyroid_t', 'query_hyperthyroid_f',
       'query_hyperthyroid_t', 'pregnant_f', 'pregnant_t', 'sick_f', 'sick_t',
       'tumor_f', 'tumor_t', 'lithium_f', 'lithium_t', 'goitre_f', 'goitre_t',
       'TSH_measured_n', 'TSH_measured_y', 'T3_measured_n', 'T3_measured_y',
       'TT4_measured_n', 'TT4_measured_y', 'T4U_measured_n', 'T4U_measured_y',
       'FTI_measured_n', 'FTI_measured_y', 'TBG_measured_n', 'TBG_measured_y'],
      dtype='object')

In [19]:
data.isnull().sum()

status                         0
age                            0
TSH                            0
T3                             0
TT4                            0
T4U                            0
FTI                            0
sex_F                          0
sex_M                          0
on_thyroxine_f                 0
on_thyroxine_t                 0
query_on_thyroxine_f           0
query_on_thyroxine_t           0
on_antithyroid_medication_f    0
on_antithyroid_medication_t    0
thyroid_surgery_f              0
thyroid_surgery_t              0
query_hypothyroid_f            0
query_hypothyroid_t            0
query_hyperthyroid_f           0
query_hyperthyroid_t           0
pregnant_f                     0
pregnant_t                     0
sick_f                         0
sick_t                         0
tumor_f                        0
tumor_t                        0
lithium_f                      0
lithium_t                      0
goitre_f                       0
goitre_t  

In [20]:
data['status'].unique()

[hypothyroid, negative]
Categories (2, object): [hypothyroid, negative]

In [0]:
predictors = ['age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI', 'sex_F', 'sex_M',
       'on_thyroxine_f', 'on_thyroxine_t', 'query_on_thyroxine_f',
       'query_on_thyroxine_t', 'on_antithyroid_medication_f',
       'on_antithyroid_medication_t', 'thyroid_surgery_f', 'thyroid_surgery_t',
       'query_hypothyroid_f', 'query_hypothyroid_t', 'query_hyperthyroid_f',
       'query_hyperthyroid_t', 'pregnant_f', 'pregnant_t', 'sick_f', 'sick_t',
       'tumor_f', 'tumor_t', 'lithium_f', 'lithium_t', 'goitre_f', 'goitre_t',
       'TSH_measured_n', 'TSH_measured_y', 'T3_measured_n', 'T3_measured_y',
       'TT4_measured_n', 'TT4_measured_y', 'T4U_measured_n', 'T4U_measured_y',
       'FTI_measured_n', 'FTI_measured_y', 'TBG_measured_n', 'TBG_measured_y']

In [0]:
#we now separate the target and predictor variables
y = data['status']
X = data[predictors]

In [0]:
#we split them into train test split
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state = 100)

In [0]:
#we create the random classifier object
forest = RandomForestClassifier(n_estimators = 100, random_state=42, min_samples_split = 20, max_depth=5)


In [25]:
#we fit the model 
forest.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=20,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
#making predictions
y_pred = forest.predict(X_test)

In [27]:
#lets evaluate our model
print('Accuracy', metrics.accuracy_score(y_test, y_pred))

Accuracy 0.9810326659641728


our model has yielded an accuracy of 98%

In [28]:
confusion_matrix = confusion_matrix(y_test,y_pred)
confusion_matrix

array([[ 27,  15],
       [  3, 904]])

In [29]:
#we now want to establish the most important feature and specifically use them to create our second model then compare
#the accuracies. theoretically we expect our second model  to yield better results.
#we first generate the array for the importances
forest.feature_importances_

array([1.42765751e-02, 3.17039254e-01, 7.82596113e-02, 2.14537369e-01,
       2.30285998e-02, 3.06003868e-01, 7.68288226e-04, 1.58472883e-03,
       1.77994237e-03, 1.68337754e-03, 1.36817664e-04, 1.92877571e-04,
       3.26809986e-03, 2.27103785e-03, 1.66186970e-03, 1.28193760e-03,
       3.14191097e-03, 4.56727569e-03, 1.86418591e-04, 3.62498993e-04,
       2.34190227e-04, 9.42845988e-04, 1.66042347e-04, 3.03785332e-03,
       1.87814671e-05, 2.89268347e-05, 0.00000000e+00, 0.00000000e+00,
       6.12452356e-04, 9.98322354e-04, 4.63188966e-03, 1.61498699e-03,
       1.17059210e-03, 1.34241034e-03, 4.64358046e-04, 2.29036219e-04,
       8.37422122e-05, 5.81796767e-05, 1.44881101e-04, 9.49389168e-04,
       3.54176132e-03, 3.69699950e-03])

In [30]:
#lets list the columns
predictors

['age',
 'TSH',
 'T3',
 'TT4',
 'T4U',
 'FTI',
 'sex_F',
 'sex_M',
 'on_thyroxine_f',
 'on_thyroxine_t',
 'query_on_thyroxine_f',
 'query_on_thyroxine_t',
 'on_antithyroid_medication_f',
 'on_antithyroid_medication_t',
 'thyroid_surgery_f',
 'thyroid_surgery_t',
 'query_hypothyroid_f',
 'query_hypothyroid_t',
 'query_hyperthyroid_f',
 'query_hyperthyroid_t',
 'pregnant_f',
 'pregnant_t',
 'sick_f',
 'sick_t',
 'tumor_f',
 'tumor_t',
 'lithium_f',
 'lithium_t',
 'goitre_f',
 'goitre_t',
 'TSH_measured_n',
 'TSH_measured_y',
 'T3_measured_n',
 'T3_measured_y',
 'TT4_measured_n',
 'TT4_measured_y',
 'T4U_measured_n',
 'T4U_measured_y',
 'FTI_measured_n',
 'FTI_measured_y',
 'TBG_measured_n',
 'TBG_measured_y']

In [31]:
#we create a dataframe of the features and their importances
output = pd.DataFrame({'features':predictors, 'importances':forest.feature_importances_}).sort_values(by='importances', ascending=False)
output


,features,importances
1,TSH,0.317039
5,FTI,0.306004
3,TT4,0.214537
2,T3,0.078260
4,T4U,0.023029
0,age,0.014277
30,TSH_measured_n,0.004632
17,query_hypothyroid_t,0.004567
41,TBG_measured_y,0.003697
40,TBG_measured_n,0.003542


from the above dataframe the top most important features were TSH, FTI,TT4

# **Gradient Boost for Classification**

In [0]:
#Lets now try a second model
from sklearn.ensemble import GradientBoostingClassifier
gradient = GradientBoostingClassifier()
target_number_of_trees = 100

In [0]:
#we fit our model in the data
gradient = GradientBoostingClassifier(n_estimators=target_number_of_trees, learning_rate=0.1, max_depth=3)
gradient = gradient.fit(X_train,y_train)

In [0]:
y_pred_grad = gradient.predict(X_test)

In [35]:
#lets evaluate the model using accuracy score
from sklearn import metrics
print('Accuracy',metrics.accuracy_score(y_test,y_pred_grad))

Accuracy 0.9884088514225501


In [36]:
#lets check the feature importances
gradient.feature_importances_

array([1.24303821e-02, 2.12899943e-01, 2.00907459e-02, 1.42231322e-02,
       2.71556331e-03, 6.82518897e-01, 1.34110739e-05, 0.00000000e+00,
       5.82018131e-03, 4.95288771e-03, 0.00000000e+00, 0.00000000e+00,
       6.11119926e-03, 7.81557829e-03, 7.66371399e-03, 4.84700581e-03,
       9.53545788e-03, 6.98196732e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.82697933e-04, 7.70692497e-05, 3.31542094e-04, 2.74529073e-04,
       1.94120550e-04, 3.19974611e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

In [37]:
#we get the column features we used in our prediction
PREDICTORS = predictors
predictors

['age',
 'TSH',
 'T3',
 'TT4',
 'T4U',
 'FTI',
 'sex_F',
 'sex_M',
 'on_thyroxine_f',
 'on_thyroxine_t',
 'query_on_thyroxine_f',
 'query_on_thyroxine_t',
 'on_antithyroid_medication_f',
 'on_antithyroid_medication_t',
 'thyroid_surgery_f',
 'thyroid_surgery_t',
 'query_hypothyroid_f',
 'query_hypothyroid_t',
 'query_hyperthyroid_f',
 'query_hyperthyroid_t',
 'pregnant_f',
 'pregnant_t',
 'sick_f',
 'sick_t',
 'tumor_f',
 'tumor_t',
 'lithium_f',
 'lithium_t',
 'goitre_f',
 'goitre_t',
 'TSH_measured_n',
 'TSH_measured_y',
 'T3_measured_n',
 'T3_measured_y',
 'TT4_measured_n',
 'TT4_measured_y',
 'T4U_measured_n',
 'T4U_measured_y',
 'FTI_measured_n',
 'FTI_measured_y',
 'TBG_measured_n',
 'TBG_measured_y']

In [38]:
#we create a dataframe that lists the features sorted by their importances
grad_outcome = pd.DataFrame({'features':PREDICTORS, 'importance_grad': gradient.feature_importances_}).sort_values(by='importance_grad',ascending=False)
grad_outcome

,features,importance_grad
5,FTI,0.682519
1,TSH,0.212900
2,T3,0.020091
3,TT4,0.014223
0,age,0.012430
16,query_hypothyroid_f,0.009535
13,on_antithyroid_medication_t,0.007816
14,thyroid_surgery_f,0.007664
17,query_hypothyroid_t,0.006982
12,on_antithyroid_medication_f,0.006111
